In [1]:
import importlib
import random

import mysklearn.myutils as myutils
import mysklearn.plotutils as plotutils
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers as myclassifiers
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyNaiveBayesClassifier

import mysklearn.myevaluation as myevaluation

In [ ]:
spotify_table = MyPyTable()
spotify_table.load_from_file("./input_data/spotify_tracks.csv")
print("Number of instances:", len(spotify_table.data))

rows_to_remove = []
genre_row = spotify_table.get_column("track_genre")
genres = myutils.get_unique_values(genre_row)
print(genres)
for i in range(len(spotify_table.data)):
    if spotify_table.data[i][spotify_table.column_names.index("speechiness")] >= 0.85 and spotify_table.data[i][-1] in ["kids", "children", "comedy"]:
        rows_to_remove.append(i)
spotify_table.drop_rows(rows_to_remove)
print("number of instances with stories removed:", len(spotify_table.data))

In [ ]:
rows_to_remove = []
track_ids = []
for i in range(len(spotify_table.data)):
    if spotify_table.data[i][1] not in track_ids:
        track_ids.append(spotify_table.data[i][1])
    else:
        rows_to_remove.append(i)
spotify_table.drop_rows(rows_to_remove)
print("Number of instances with duplicates removed:", len(spotify_table.data))

In [ ]:
popularity_index_list = []
popularity_column = spotify_table.get_column("popularity")
for i in range(len(popularity_column)):
    popularity_index_list.append([popularity_column[i], i])
popularity_index_list = sorted(popularity_index_list)

rows_to_remove = [instance[1] for instance in popularity_index_list]
rows_to_remove = rows_to_remove[:(len(popularity_index_list) - 10000)]
spotify_table.drop_rows(rows_to_remove)
print("Final number of instances:", len(spotify_table.data))
print(spotify_table.data[:10])

In [2]:
spotify_table = MyPyTable()
spotify_table.load_from_file("./input_data/cleaned_spotify_tracks.csv")

Once we narrowed our data down to the 10,000 most ppular songs, we looked at the ways it needed to be cleaned before classification. Track_genre was the only column that seemed to need cleaning, so we tried a few ways to clean track_genre to make it useable for the classifiers. There were so many unique values and some had so many more instances than others that it was difficult to find a logical way to regroup them while maintaining a reasonable distribution. Ultimately, we decided that track_genre probably wouldn't even be a good enough predictor for it to be worth the effort it would take to properly clean the data.

We ran our knn classifier on our 10,000 instance dataset, and it took about 30 minutes to come up with a model. Because it took so long with so many instances, we decided to cut our data down to only 1,000 instances using random sampling. We then got rid of the index column and the track id column because they no longer added any information to our data.

In [3]:
spotify_table.data = random.sample(spotify_table.data, k=1000)

for i in range(len(spotify_table.data)):
    spotify_table.data[i] = spotify_table.data[i][2:]
spotify_table.column_names = spotify_table.column_names[2:]
print(spotify_table.column_names, spotify_table.data[:10])

['artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre'] [['Aimer', 'daydream', 'カタオモイ', 67.0, 207360.0, 'False', 0.621, 0.533, 1.0, -5.974, 1.0, 0.0342, 0.78, 0.0, 0.333, 0.791, 96.981, 4.0, 'anime'], ['Shawn Mendes', 'Handwritten', 'Stitches', 80.0, 206880.0, 'False', 0.746, 0.754, 1.0, -6.684, 1.0, 0.0676, 0.0152, 0.0, 0.0486, 0.746, 149.882, 4.0, 'dance'], ['M-22', 'Need You There', 'Need You There', 63.0, 185991.0, 'False', 0.651, 0.76, 0.0, -5.949, 1.0, 0.0446, 0.0273, 0.000538, 0.315, 0.137, 122.969, 4.0, 'deep-house'], ['Jeff Bernat', 'She Loves Me Not', 'Still', 65.0, 226926.0, 'False', 0.485, 0.315, 4.0, -11.599, 1.0, 0.0946, 0.581, 0.000349, 0.11, 0.425, 134.714, 4.0, 'chill'], ['Casting Crowns;Matthew West', 'Only Jesus', 'Nobody (feat. Matthew West)', 67.0, 206400.0, 'False', 0.561, 

After removing those columns, we tried a few ways to clean track_genre to make it useable for the classifiers. There were so many unique values and some had so many more instances than others that it was difficult to find a logical way to regroup them while maintaining a reasonable distribution. Ultimately, we decided that track_genre probably wouldn't even be a good enough predictor for it to be worth the effort it would take to properly clean the data.

In [4]:
spotify_table.data = random.sample(spotify_table.data, k=1000)

In [5]:
spotify_table.save_to_file("input_data/sampled_cleaned_spotify_tracks.csv")

df = pd.dataframe(spotify_table.data)
df["col"] = dr["col"].astype(int)

#clean all with fewer than 5 of each classification
get rid of id row, track id?
most popular genres, and all less popular grouped into "other"